In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Загрузка CSV-файла
df = pd.read_csv('data4.4b.csv', sep=';')

# Заполняем пустые значения и преобразуем всё в строки
df = df.fillna('').astype(str)

# Извлекаем уникальные имена персонажей
all_characters = pd.unique(df.values.ravel())  # Получаем всех персонажей как массив
unique_characters = set()

for team in all_characters:
    if team.strip():  # Игнорируем пустые строки
        unique_characters.update(team.split(';'))  # Разделяем отряды на индивидуальные имена

unique_characters = sorted(unique_characters)  # Сортируем для согласованности

# Обучаем LabelEncoder
encoder = LabelEncoder()
encoder.fit(unique_characters)

# Преобразуем команды в числовые индексы
encoded_teams = []
for _, row in df.iterrows():
    team = [encoder.transform([char])[0] for char in row if char.strip()]
    encoded_teams.append(team)

# Создаем входные (X) и выходные (y) данные
X = []
y = []

# Для каждого персонажа в команде создаем вход и выход (предсказание 3 других персонажей)
for team in encoded_teams:
    for character in team:
        X.append([character])  # Вход - один персонаж
        other_chars = [ch for ch in team if ch != character]
        y.append(other_chars[:3])  # Выход - 3 персонажа

X = np.array(X)

# Преобразуем y в объектный тип, чтобы хранить списки разной длины
y = np.array(y, dtype=object)

print("X shape:", X.shape)
print("y shape:", y.shape)

def convert_to_multiclass(y):
    y_multiclass = []
    for team in y:
        multiclass_vector = np.zeros(len(encoder.classes_))
        for char_index in team:
            multiclass_vector[char_index] = 1  
        y_multiclass.append(multiclass_vector)
    return np.array(y_multiclass)
y_multiclass = convert_to_multiclass(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_multiclass, test_size=0.2, random_state=42)


X shape: (49635, 1)
y shape: (49635,)


In [20]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

def convert_to_multiclass(y):
    y_multiclass = []
    for team in y:
        multiclass_vector = np.zeros(len(encoder.classes_))
        for char_index in team:
            multiclass_vector[char_index] = 1  
        y_multiclass.append(multiclass_vector)
    return np.array(y_multiclass)
y_multiclass = convert_to_multiclass(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_multiclass, test_size=0.2, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(encoder.classes_), output_dim=64, input_length=1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(encoder.classes_), activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)

input_character = "Furina"  
input_index = encoder.transform([input_character])[0]
predicted_probs = model.predict(np.array([[input_index]])) 
predicted_indices = np.argsort(predicted_probs[0])[-3:][::-1]  

predicted_characters = encoder.inverse_transform(predicted_indices)
print("Предсказанные персонажи:", predicted_characters)


c:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.1290 - loss: 0.1982 - val_accuracy: 0.2199 - val_loss: 0.1089
Epoch 2/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.2069 - loss: 0.1091 - val_accuracy: 0.2064 - val_loss: 0.1055
Epoch 3/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.2087 - loss: 0.1066 - val_accuracy: 0.2034 - val_loss: 0.1048
Epoch 4/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.2049 - loss: 0.1057 - val_accuracy: 0.2166 - val_loss: 0.1041
Epoch 5/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.2106 - loss: 0.1049 - val_accuracy: 0.2220 - val_loss: 0.1040
Epoch 6/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.2068 - loss: 0.1045 - val_accuracy: 0.2143 - val_loss: 0.1037
Epoch 7/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.2079 - loss: 0.1046 - val_accuracy: 0.2060 - val_loss: 0.1033
Epoch 8/20
621/621 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.2058 - loss: 0.1040 - val_accuracy: 0

In [21]:
"""
Схранение
model.save('character_recommendation_model.h5')


Загрузка
from tensorflow.keras.models import load_model

loaded_model = load_model('character_recommendation_model.h5')

"""

'\nСхранение\nmodel.save(\'character_recommendation_model.h5\')\nЗагрузка\nfrom tensorflow.keras.models import load_model\n\n# Загрузка модели\nloaded_model = load_model(\'character_recommendation_model.h5\')\nprint("Модель загружена!")\n\n'

In [23]:
model.save('character_recommendation_model.keras')

In [3]:
from tensorflow.keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint('character_recommendation_model.keras', save_best_only=True, save_weights_only=False, monitor='val_loss', mode='min')
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[checkpoint])

c:\Users\User\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 11 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


NameError: name 'X_train' is not defined

In [28]:
# Пример индексов и их соответствующих имен персонажей
sample_indices = X[:10]  # Пример индексов для первых 10 записей
decoded_names = encoder.inverse_transform(sample_indices)
print("Декодированные имена для первых 10 записей:", decoded_names)


Декодированные имена для первых 10 записей: ['Raiden Shogun' 'Kujou Sara' 'Sangonomiya Kokomi' 'Kaedehara Kazuha'
 'Beidou' 'Lisa' 'Jean' 'Xianyun' 'Raiden Shogun' 'Kujou Sara']


c:\Users\User\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
input_character = "Nahida"  
input_index = encoder.transform([input_character])[0]
predicted_probs = model.predict(np.array([[input_index]])) 
predicted_indices = np.argsort(predicted_probs[0])[-3:][::-1]  

predicted_characters = encoder.inverse_transform(predicted_indices)
print("Предсказанные персонажи:", predicted_characters)


NameError: name 'encoder' is not defined

In [5]:
# Пример предсказания
def recommend_team(input_character, model, encoder, top_k=3):
    input_index = encoder.transform([input_character])[0]  # Кодируем входного персонажа
    prediction = model.predict(np.array([input_index]))[0]  # Получаем предсказание

    # Получаем `top_k` персонажей
    recommended_indices = np.argsort(prediction)[-top_k:][::-1]
    recommended_characters = encoder.inverse_transform(recommended_indices)
    return recommended_characters

# Пример использования
input_character = "Furina"
recommended_team = recommend_team(input_character, model, encoder)
print(f"Для персонажа {input_character} рекомендуется команда: {recommended_team}")


NameError: name 'model' is not defined

In [4]:
import tensorflow as tf

# Загрузка сохранённой модели
model = tf.keras.models.load_model('character_recommendation_model.keras')

# Проверка, что модель загружена
model.summary()


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)        │ (None, 1, 64)          │         5,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 512)            │        33,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 79)             │        10,191 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 425,504 (1.62 MB)

 Trainable params: 212,751 (831.06 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 212,753 (831.07 KB)

In [5]:
input_character = "Hu Tao"  # Пример входного персонажа
input_index = encoder.transform([input_character])[0]

# Получение предсказаний
predicted_probs = model.predict(np.array([[input_index]]))  # Вход - индекс персонажа
predicted_indices = np.argsort(predicted_probs[0])[-3:][::-1]
predicted_characters = encoder.inverse_transform(predicted_indices)
print("Предсказанные персонажи:", predicted_characters)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
Предсказанные персонажи: ['Yelan' 'Xingqiu' 'Furina']


In [6]:
# Проверка точности
loss, accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)

# Продолжение обучения
model.fit(X_train, y_train, epochs=500, batch_size=64, validation_data=(X_test, y_test))


311/311 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2045 - loss: 0.1015
Test accuracy: 0.2084214836359024
Epoch 1/500
621/621 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.2125 - loss: 0.1013 - val_accuracy: 0.2078 - val_loss: 0.1017
Epoch 2/500
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.2100 - loss: 0.1013 - val_accuracy: 0.2080 - val_loss: 0.1017
Epoch 3/500
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.2157 - loss: 0.1012 - val_accuracy: 0.2073 - val_loss: 0.1017
Epoch 4/500
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.2133 - loss: 0.1009 - val_accuracy: 0.2063 - val_loss: 0.1017
Epoch 5/500
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.2128 - loss: 0.1013 - val_accuracy: 0.2129 - val_loss: 0.1017
Epoch 6/500
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.2099 - loss: 0.1014 - val_accuracy: 0.2129 - val_loss: 0.1017
Epoch 7/500
621/621 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.2141 - loss: 0.1012 - val_accuracy: 0.2101 - val_l

In [7]:
model.save('character_recommendation_model.keras')
